In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [419 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [863 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,085 kB]
Get:13 htt

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.createOrReplaceTempView("home_sales")
# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, round

df = df.withColumn("price", col("price").cast("double"))

# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
fourbed_avg_price_by_year = df.filter(col("bedrooms") == 4).groupBy(year("date").alias("year")).avg("price")

# Round the average price to two decimal places
fourbed_avg_price_by_year = fourbed_avg_price_by_year.withColumn("avg(price)", round(col("avg(price)"), 2))

# Show the result
fourbed_avg_price_by_year.show()



+----+----------+
|year|avg(price)|
+----+----------+
|2022| 296363.88|
|2019|  300263.7|
|2020| 298353.78|
|2021| 301819.44|
+----+----------+



In [7]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [17]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
threebedbath_avg_price_by_year =spark.sql("""
 SELECT
 date_built AS date_built,
 ROUND(AVG(price), 2) AS avg_price
 FROM home_sales
 WHERE bedrooms = 3
 AND bathrooms = 3
 GROUP BY date_built
 ORDER BY date_built""")

threebedbath_avg_price_by_year.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [15]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price =spark.sql("""
 SELECT
 date_built AS date_built,
 ROUND(AVG(price), 2) AS avg_price
 FROM home_sales
 WHERE bedrooms = 3
 AND bathrooms = 3
 AND floors = 2
 AND sqft_living >=2000
 GROUP BY date_built
 ORDER BY date_built""")

avg_price.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [ ]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [23]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_ratng = spark.sql("""
SELECT
ROUND(AVG(price), 2) AS home,
view
FROM home_sales
WHERE price >= 350000
GROUP BY view""")

view_ratng.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----------+----+
|      home|view|
+----------+----+
| 403005.77|   7|
| 788128.21|  51|
|  404673.3|  15|
| 798684.82|  54|
| 399548.12|  11|
| 397771.65|  29|
| 750537.94|  69|
|  396964.5|  42|
| 1072285.2|  87|
| 752861.18|  73|
| 767036.67|  64|
|  398867.6|   3|
|  397862.0|  30|
| 401419.75|  34|
|  791453.0|  59|
| 398592.71|   8|
| 402124.62|  28|
| 402022.68|  22|
|1056336.74|  85|
| 399586.53|  16|
+----------+----+
only showing top 20 rows

--- 0.959505558013916 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [34]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Query 1: Using the cached data
start_time = time.time()
df.createOrReplaceTempView("home_sales_cached")
# Run the query on the cached DataFrame
view_rating_avg_price_cached = spark.sql("""
    SELECT
    view,
    ROUND(AVG(price), 2) AS home
    FROM home_sales_cached
    WHERE price >= 350000
    GROUP BY view""")
view_rating_avg_price_cached.show()
# Calculate the runtime for the cached query
cached_runtime = time.time() - start_time

print("--- Cached Runtime: %s seconds ---" % cached_runtime)

# Uncache the DataFrame
spark.table("home_sales").unpersist()
df.createOrReplaceTempView("home_sales")
# Query 2: Using the uncached data
start_time = time.time()

# Run the same query on the original DataFrame without caching
view_rating_avg_price_uncached = spark.sql("""
    SELECT
    view,
    ROUND(AVG(price), 2) AS home
    FROM home_sales_cached
    WHERE price >= 350000
    GROUP BY view""")
view_rating_avg_price_uncached.show()
# Calculate the runtime for the uncached query
uncached_runtime = time.time() - start_time

print("--- Uncached Runtime: %s seconds ---" % uncached_runtime)




+----+----------+
|view|      home|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- Cached Runtime: 0.7790875434875488 seconds ---
+----+----------+
|view|      home|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- Uncached Runtime: 0.807988882

In [35]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [36]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("/content/gdrive/My Drive/Home_Sales_Colab1/")



In [37]:
# 11. Read the parquet formatted data.
df_parquet = spark.read.parquet("/content/gdrive/My Drive/Home_Sales_Colab1/")

In [38]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("parquet_data")

In [39]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

# Run the query on the parquet DataFrame
view_rating_avg_price_parquet = spark.sql("""
    SELECT
    view,
    ROUND(AVG(price), 2) AS home
    FROM parquet_data
    WHERE price >= 350000
    GROUP BY view""")

view_rating_avg_price_parquet.show()

# Calculate the runtime for the parquet query
parquet_runtime = time.time() - start_time
print("--- Parquet Runtime: %s seconds ---" % parquet_runtime)

start_time = time.time()
df.createOrReplaceTempView("home_sales_cached")

# Run the query on the cached DataFrame
view_rating_avg_price_cached = spark.sql("""
    SELECT
    view,
    ROUND(AVG(price), 2) AS home
    FROM home_sales_cached
    WHERE price >= 350000
    GROUP BY view""")

view_rating_avg_price_cached.show()

# Calculate the runtime for the cached query
cached_runtime = time.time() - start_time
print("--- Cached Runtime: %s seconds ---" % cached_runtime)

# Compare the runtimes
if parquet_runtime < cached_runtime:
    print("Parquet query is faster than the cached version.")
else:
    print("Cached version is faster than the Parquet query.")

+----+----------+
|view|      home|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  73| 752861.18|
|  87| 1072285.2|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  35| 401934.21|
+----+----------+
only showing top 20 rows

--- Parquet Runtime: 1.5317063331604004 seconds ---
+----+----------+
|view|      home|
+----+----------+
|   7| 403005.77|
|  51| 788128.21|
|  15|  404673.3|
|  54| 798684.82|
|  11| 399548.12|
|  29| 397771.65|
|  69| 750537.94|
|  42|  396964.5|
|  87| 1072285.2|
|  73| 752861.18|
|  64| 767036.67|
|   3|  398867.6|
|  30|  397862.0|
|  34| 401419.75|
|  59|  791453.0|
|   8| 398592.71|
|  28| 402124.62|
|  22| 402022.68|
|  85|1056336.74|
|  16| 399586.53|
+----+----------+
only showing top 20 rows

--- Cached Runtime: 1.3318979740

In [40]:
# 14. Uncache the home_sales temporary table.
spark.table("home_sales").unpersist()

DataFrame[id: string, date: string, date_built: string, price: double, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [41]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False